# Laboratorio 6.a

Ranking su basicness dei synset di WordNet. Si richiede di creare un mapping tra i synset di WordNet ed un basicness score (ad es. [0, 1]), utilizzando dati, risorse, features e approcci che credete opportuni alla risoluzione del task. 

ALTRA IDEA: Dato un synset, guardare se esiste un iponimo/iperonimo/contrario ecc nel dataset. es. trovo cane e tra gli iponimi c'è shitzu, e shitzu c'è nel dataset ed è etichettato come advanced, allora posso dare un incremento allo score di cane

In [224]:
import json
import pandas as pd
from nltk.corpus import wordnet as wn
from scipy.stats import pearsonr
from scipy.stats import spearmanr

## Apertura del file

In [225]:
def open_json(file_name):
    with open(file_name, 'r') as file:
        data = json.load(file)

    df = pd.DataFrame(data['dataset'], columns=['dataset'])

    df[['synset','words','gloss']] = df['dataset'].str.split(':',expand=True)
    df = df.drop(columns=['dataset'])

    #remove from synset the substrings "Synset('" and "')"
    df['synset'] = df['synset'].str.replace('Synset\(\'', '', regex=True)
    df['synset'] = df['synset'].str.replace('\'\)', '', regex=True)
    df['words'] = df['words'].str.replace(' \| Definition', '', regex=True)

    df['isHard'] = data['isHard']
    df['answers'] = data['answers']
    df['timeDiffs'] = data['timeDiffs']

    df['basicness1'] = None
    df['basicness2'] = None

    return df

## Metodi di supporto

Metodi personali per estrazione di informazioni da WordNet

In [226]:
# calcolo la profondità massima del grafo
MAX_DEPTH = max(max(len(hyp_path) for hyp_path in ss.hypernym_paths()) for ss in wn.all_synsets())

In [235]:
'''
dato un synset, restituisce la profondità del synset nella geerarchia di WordNet
'''
def get_hyponyms(synset):
    hyponyms = set()
    for hyponym in synset.hyponyms():
        hyponyms |= get_hyponyms(hyponym)
    return hyponyms | set(synset.hyponyms())


def get_hyponyms_paths(synset, max_depth, current_depth=0):
    if current_depth >= max_depth:
        return []

    hyponym_paths = []
    for hyponym in synset.hyponyms():
        paths_from_hyponym = get_hyponyms_paths(hyponym, max_depth, current_depth + 1)
        if paths_from_hyponym:
            for path in paths_from_hyponym:
                hyponym_paths.append([synset] + path)
        else:
            hyponym_paths.append([synset, hyponym])

    return hyponym_paths


def get_max_hyponym_paths(synset):
    paths = get_hyponyms_paths(synset, MAX_DEPTH)
    if paths == []:
        return 0

    max_length = max(len(path) for path in paths)

    return max_length

In [228]:
'''
Calcola il percorso più breve dalla radice al senso. Alcune volte synset.hypernyms() non restituiva alcun iperonimo, 
quindi, guardando dall'implementazione del metodo min_depth di nltk è stato inserito negli iperonimi il risulato 
di synset.instance_hypernyms()
'''
def get_min_depth(synset):
    if not synset:
        return 0
    
    hyperonyms = synset.hypernyms() + synset.instance_hypernyms()
    if not hyperonyms:
        return 0
    else:
        depths = []
        for hyp in hyperonyms:
            dep = 1 + get_min_depth(hyp)
            depths.append(dep)
        return min(depths)

## Creazione del Basicness scoring

In [229]:
def compute_basicness_score1(synset):
    min_depth = get_min_depth(synset)

    basicness = 1 - (min_depth / MAX_DEPTH)

    #print(synset, ': ', min_depth, ' /',MAX_DEPTH ,':', basicness)

    return basicness

'''
Tengo in considerazione che i synset possono provenire da diversi sottografi collegati a root
'''
def compute_basicness_score2(synset):
    min_depth = get_min_depth(synset)
    max_hyponym_paths = get_max_hyponym_paths(synset)

    if min_depth == 0:
        basicness = 1
    else:
        basicness = 1 - (min_depth / (min_depth + max_hyponym_paths))

    #print(synset, ': ', min_depth, ' / (', min_depth, ' + ', max_hyponym_paths, '):', basicness)

    return basicness

## MAIN

### Apro il file

In [230]:
file_name = "basic_advanced/1.json"

df = open_json(file_name)
df.head()

,synset,words,gloss,isHard,answers,timeDiffs,basicness1,basicness2
0,war.n.01,"war, warfare",the waging of armed conflict against an enemy,False,basic,3.341,None,None
1,fiefdom.n.01,fiefdom,the domain controlled by a feudal lord,False,advanced,1.333,None,None
2,bed.n.03,"bed, bottom",a depression forming the ground under a body ...,False,basic,0.968,None,None
3,return_on_invested_capital.n.01,"return on invested capital, return on investme...","(corporate finance) the amount, expressed as ...",False,advanced,2.315,None,None
4,texture.n.02,texture,the essential quality of something,True,basic,7.346,None,None


### Calcolo le basicness con le due metodologie implementate per ogni synset

In [231]:
for index, row in df.iterrows():
    synset = wn.synset(row['synset'])
    df.at[index, 'basicness1'] = compute_basicness_score1(synset)
    df.at[index, 'basicness2'] = compute_basicness_score2(synset)

df_basic = df[df['answers'] == 'basic']
df_advanced = df[df['answers'] == 'advanced']

### Calcolo degli indici di correlazione

Il coefficiente di correlazione di Spearman e il coefficiente di correlazione di Pearson sono due misure statistiche utilizzate per valutare la relazione tra due variabili. Tuttavia, la principale differenza tra i due è il tipo di dati che possono essere trattati e la natura delle relazioni che misurano.

Il coefficiente di correlazione di Pearson, noto anche come correlazione lineare, valuta la relazione lineare tra due variabili continue. Misura la forza e la direzione della relazione lineare tra le variabili e può assumere valori compresi tra -1 e 1. Un valore di 1 indica una correlazione positiva perfetta, -1 indica una correlazione negativa perfetta e 0 indica l'assenza di correlazione lineare.

Il coefficiente di correlazione di Spearman, d'altra parte, valuta la relazione monotona tra due variabili, che può essere di qualsiasi tipo di scala. Non richiede che le variabili seguano una relazione lineare, ma solo che la relazione sia monotona. Questo coefficiente misura la forza e la direzione della relazione monotona e può assumere valori compresi tra -1 e 1. Una correlazione di Spearman di 1 indica una correlazione monotona positiva perfetta, -1 indica una correlazione monotona negativa perfetta e 0 indica l'assenza di correlazione monotona.

In sintesi, il coefficiente di correlazione di Pearson valuta la relazione lineare tra due variabili continue, mentre il coefficiente di correlazione di Spearman valuta la relazione monotona tra due variabili, che può essere di qualsiasi tipo di scala. Entrambi i coefficienti sono utili strumenti statistici per analizzare le relazioni tra variabili, ma la scelta tra i due dipenderà dalle caratteristiche delle variabili e dalla natura della relazione che si desidera esaminare.

#### Calcolo l'indice di correlazione di Pearson

In [232]:
corr, _ = pearsonr(df_basic['basicness1'], df_basic['timeDiffs'])
print('Pearsons correlation basicness1: %.3f' % corr)

corr, _ = pearsonr(df_basic['basicness2'], df_basic['timeDiffs'])
print('Pearsons correlation basicness2: %.3f' % corr)

Pearsons correlation basicness1: -0.011
Pearsons correlation basicness2: 0.037


#### Calcolo l'indice di correlazione di Spaerman

In [233]:
corr, _ = spearmanr(df_basic['basicness1'], df_basic['timeDiffs'])
print('Spearmans correlation basicness1: %.3f' % corr)

corr, _ = spearmanr(df_basic['basicness2'], df_basic['timeDiffs'])
print('Pearsons correlation basicness2: %.3f' % corr)

Spearmans correlation basicness1: -0.122
Pearsons correlation basicness2: -0.167


### Mostro i valori medi per le parole basic e advance per le due tipologie di basicness calcolate

In [234]:
# valori medi di basicness1 e basicness2 per df_basic e df_advanced
print('basicness1 mean basic: ', df_basic['basicness1'].mean())
print('basicness1 mean advanced: ', df_advanced['basicness1'].mean())

print('\n')

print('basicness2 mean basic: ', df_basic['basicness2'].mean())
print('basicness2 mean advanced: ', df_advanced['basicness2'].mean())

basicness1 mean basic:  0.6801038062283739
basicness1 mean advanced:  0.619767441860465


basicness2 mean basic:  0.29368658542533976
basicness2 mean advanced:  0.06955358594893479
